# Combined hypothesis testing

This notebook combines and integrates the entire analysis workflow. It tests whether
- there was an increase in potential reproducibility in Agile papers after the introduction of the guidelines in 2020 (A)
- there was an increase in potential reproducibilit in GIScience papers after that introduction (B)
- there was a difference in increase between the two conference series (C)

## Preliminaries: preparing data and analysis functions

This sections loads and prepares the input data, and provides the functions necessary for the hypothesis testing. It only needs to be run once. All later sections depend on this one, but are independent from one another. That means you can explore different variable settings for the hypotheses without interfering with each other. 

### Setup and prepare data
You can set global variables after the necessary libraries are loaded, then the data is loaded and preprocessed. 

The script assumes that the available data is a CSV file with the following relevant columns and variable values (for detailed explanations of the UDAO scheme, see the corresponding paper). 

- conf: "agile" or "giscience"
- year: year of the conference proceedings
- consolidated_cp: conceptual paper true/false
- consolidated_data: data dimension in UDAO (undocumented, documented, available, open) scheme
- consolidated_methods: methods dimension in UDAO scheme
- consolidated_results: results dimension in UDAO scheme
- consolidated_ce: computational environment true/false

The preprocessing requires the same two steps for all hypotheses:

1. We need to remove all conceptual papers.
2. For basic calculations on ranks (see discussion on that in a later section), we need to convert the UDAO scheme into ranks 1 to 4.

In [1]:
# all imports
import pandas as pd
import numpy as np
from scipy.stats import mannwhitneyu
from statsmodels.stats.contingency_tables import Table2x2
from IPython.display import display

pd.set_option('future.no_silent_downcasting', True)
pd.options.display.float_format = "{:.3f}".format


# change variables as needed
INPUT_CSV = '../data-clean/all-data.csv'
CONF_COL = 'conf' # column that contains the two conferences (or main groups to be compared)
GROUP_A = 'agile' # the value of CONF_COL to use for first main group
GROUP_B = 'giscience' # the value of CONF_COL to use for second main group
YEAR_COL = 'year' # column that contains the year of the conference (for creating secondary groups pre-/post-intervention)
GROUP_A1 = [2017, 2018, 2019] # years to use for creating secondary group pre-intervention from first main group
GROUP_A2 = [2021, 2022, 2023,2024] # years to use for creating secondary group post-intervention from first main group
GROUP_B1 = [2016, 2018] # years to use for creating secondary group pre-intervention from second main group
GROUP_B2 = [2021, 2023] # years to use for creating secondary group post-intervention from second main group
FILTER_COL = 'consolidated_cp' # column name used for filtering, e.g., conceptual papers; BOOLEAN mask
TEST_COL = ['consolidated_data', 'consolidated_methods', 'consolidated_results'] # column to be read and converted
CRITERIA = ['data', 'methods', 'results'] # for creating new columns with converted values of TEST_COL, has to be same length as TEST_COL
REPLACE = {
    'Not applicable': np.nan,
    'U': 0,
    'D': 1,
    'A': 2,
    'O': 3
}
P_VALUE=0.01

In [2]:
# Load the CSV file
df = pd.read_csv(INPUT_CSV)

# remove conceptual papers
mask = df[FILTER_COL] == True
filtered_df = df[~mask].copy()

print(len(df), len(filtered_df))

224 203


In [3]:
# convert values
for i in range(len(TEST_COL)):
    filtered_df.loc[:, CRITERIA[i]] = filtered_df.loc[:, TEST_COL[i]].replace(REPLACE).astype('Int64')

filtered_df.head()

,conf,paper,year,title,link,oa,dasa,rev1,rev2,rev1_cp,...,consolidated_results,consolidated_ce,consolidated_notes,disagr_type,agile_badge,agile_reproreport,disagr_id,data,methods,results
0,agile,agile_2017_006,2017,Follow the Signs—Countering Disengagement from...,https://link.springer.com/chapter/10.1007/978-...,no,no,FO,AG,False,...,D,False,no disagreement,no disagreement,NaN,NaN,0,1,1,1
1,agile,agile_2017_014,2017,The Effect of Regional Variation and Resolutio...,https://link.springer.com/chapter/10.1007/978-...,no,no,FO,AG,False,...,D,False,no disagreement,no disagreement,NaN,NaN,0,1,1,1
2,agile,agile_2019_003,2019,Evaluating the Effectiveness of Embeddings in ...,https://link.springer.com/chapter/10.1007/978-...,no,no,FO,AG,False,...,D,False,rev1 correct; broken link to Twitter dataset; ...,uncertain assessment,NaN,NaN,3,2,1,1
3,agile,agile_2019_011,2019,Enhancing the Use of Population Statistics Der...,https://link.springer.com/chapter/10.1007/978-...,no,no,FO,AG,False,...,D,False,Rev2 correct,uncertain assessment,NaN,NaN,3,0,1,1
4,agile,agile_2021_007,2021,A Comparative Study of Typing and Speech For M...,https://agile-giss.copernicus.org/articles/2/7...,yes,yes,FO,AG,False,...,O,False,"Rev1 correct, everything available in on Figsh...",uncertain assessment,True,https://osf.io/p473e,3,3,3,3


### Setup analysis functions

For testing hypotheses A and B, we have

- two independent groups (pre- and post-intervention) with unequal sample sizes
- the data is ranked (i.e., a value of 4 is better than a value of 2, but not double)

The question is whether any one group has significantly higher or lower ranks than the other.

The suitable test is Mann-Whitney U (Wilcoxon rank-sum test) which

- is non-parametric
- compares sum of ranks between groups
- suited for ordinal data
- robust to different sample sizes

The analysis function takes two groups and a column to test on and returns a dataframe. A secondary analysis function for descriptive statistics takes the same input and returns also a dataframe. 

In [4]:
def mann_whitney_u(group_1, group_2, criterion):
    # Perform the Mann–Whitney U test
    stat, p_value = mannwhitneyu(group_1[criterion], group_2[criterion], alternative='two-sided')

    # Add the results to data frame
    u_stats= pd.DataFrame([
        {
            ("U-Statistic", ""): stat, 
            ("P-value", ""): p_value
        }
    ])

    return u_stats


def descr_stats(group_1, group_2, criterion):
    # Collect results in a data frame
    d_stats = pd.DataFrame([
        {
            ("Mean", "pre"): group_1[criterion].mean(),
            ("Mean", "post"): group_2[criterion].mean(),
            ("Median", "pre"): int(group_1[criterion].median()),
            ("Median", "post"): int(group_2[criterion].median()),
            ("Mode", "pre"): group_1[criterion].mode().iloc[0],
            ("Mode", "post"): group_2[criterion].mode().iloc[0]
        }
    ])

    return d_stats

For testing Hypothesis C, we have

- two independent groups (conferences)
- for each group, we have two independent sub-groups (pre- and post-intervention) with unequal sample sizes
- the data is ranked (i.e., a value of 4 is better than a value of 2, but not double)

The question is whether any difference in ranking before and after is different between the two conferences.

An often suggested approach to compare the change over time between two groups with ordinal data is Aligned Rank Transform Analysis of Variance (e.g., ARTool package in R). However, there have been raised serious concerns over its reliability (https://statransform.github.io/jovi/). 

An alternative would be to run another MannWhitneyU on the deltas between the two conference groups, but that would require quantitatively meaningful differences between ranks (e.g., a rank of 4 is twice that of 2), which we don't have (see above). 

That leaves a descriptive comparison between effect sizes, e.g., by computing the rank-biserial for both conference groups. 

In [5]:
def rank_biserial(group_1, group_2, criterion):
    # Perform the Mann–Whitney U test
    stat, p_value = mannwhitneyu(group_1[criterion], group_2[criterion], alternative='two-sided')

    # Sample sizes
    n1 = len(group_1)
    n2 = len(group_2)

    # Compute rank-biserial correlation
    r = 1 - (2 * stat) / (n1 * n2)

    # Add the results to data frame
    effect_stats= pd.DataFrame([
        {
            "U-Statistic": stat, 
            "P-value": p_value,
            "Rank-Biserial": r
        }
    ])

    return effect_stats

Another option is to compare odds ratios. For this, we need to collapse the ranks into binary "low" (0, 1) and "high" (2, 3). We define "high" as the desirable event. 

In [31]:
def odds_ratios(group_1, group_2, criterion):
    # Collapse to binary outcome: high = 3 or 4
    group_1["High"] = (group_1[criterion] >= 2).astype(int)
    group_2["High"] = (group_2[criterion] >= 2).astype(int)

    # create 2x2 contingency table
    table = pd.DataFrame(
        [[len(group_2[group_2['High']==1]), len(group_2[group_2['High']==0])],
        [len(group_1[group_1['High']==1]), len(group_1[group_1['High']==0])]],  
        index=["after", "before"],
        columns=["high", "low"]
    )
    table2x2 = Table2x2(table.values)

    print(table)
    
    # Add the results to data frame
    odds_stats= pd.DataFrame([
        {
            "OR": table2x2.log_oddsratio, 
            "CI lower": table2x2.log_oddsratio_confint(alpha=0.01)[0],
            "CI upper": table2x2.log_oddsratio_confint(alpha=0.01)[1],
            "P-value": table2x2.test_ordinal_association().pvalue
        }
    ])

    return odds_stats

## Hypothesis A

First, let's select the conference and then check the distribution over the years. Then you can specify the years to create the groups, before the dataframe is filtered and the test carried out.

In [7]:
df_A = filtered_df[filtered_df[CONF_COL] == GROUP_A]

print(df_A[YEAR_COL].value_counts().sort_index(), len(df_A))

year
2017    16
2018    17
2019    18
2020    21
2021    13
2022    20
2023    13
2024    12
Name: count, dtype: int64 130


In [8]:
# Create the two groups
group_a1 = df_A[df_A[YEAR_COL].isin(GROUP_A1)][CRITERIA].dropna()
group_a2 = df_A[df_A[YEAR_COL].isin(GROUP_A2)][CRITERIA].dropna()

In [9]:
d_stats = pd.DataFrame()
u_stats = pd.DataFrame()

for criterion in CRITERIA:
    d_stats = pd.concat([d_stats, descr_stats(group_a1, group_a2, criterion)], ignore_index=True)
    u_stats = pd.concat([u_stats, mann_whitney_u(group_a1, group_a2, criterion)], ignore_index=True)

d_stats.columns = pd.MultiIndex.from_tuples(d_stats.columns)
u_stats.columns = pd.MultiIndex.from_tuples(u_stats.columns)

hyp_A = d_stats.join(u_stats)
hyp_A.index = CRITERIA
display(hyp_A)
print('n = ' + str(len(group_a1)) + ' (Pre-Intervention) & n = ' + str(len(group_a2)) + ' (Post-Intervention)')

Mean       Median      Mode      U-Statistic P-value
          pre  post    pre post  pre post                    
data    0.745 1.586      1    2    0    2     804.000   0.000
methods 1.020 1.931      1    2    1    2     565.000   0.000
results 1.020 1.741      1    2    1    1     740.000   0.000

n = 51 (Pre-Intervention) & n = 58 (Post-Intervention)


For all three dimensions, we observe a statistically significant increase in potential reproducibility at the chosen significance level of 0.01.

We therefore reject the original hypothesis for data, methods, and results dimensions.

Further, the descriptive statistics show that for all dimension/statistic combinations (except the mode of results) the ranks have improved. Especially the increase of mode and medians for data and methods is meaningful, because improving to rank 2 ("Available") has the largest practical impact. 

## Hypothesis B

Follows the pattern of Hypothesis a: First, let's select the conference and then check the distribution over the years. Then you can specify the years to create the groups, before the dataframe is filtered and the test carried out.

In [10]:
df_B = filtered_df[filtered_df[CONF_COL] == GROUP_B]

print(df_B[YEAR_COL].value_counts().sort_index(), len(df_B))

year
2016    17
2018    17
2020    15
2021    13
2023    11
Name: count, dtype: int64 73


In [11]:
# Create the two groups
group_b1 = df_B[df_B[YEAR_COL].isin(GROUP_B1)][CRITERIA].dropna()
group_b2 = df_B[df_B[YEAR_COL].isin(GROUP_B2)][CRITERIA].dropna()

In [12]:
d_stats = pd.DataFrame()
u_stats = pd.DataFrame()

for criterion in CRITERIA:
    d_stats = pd.concat([d_stats, descr_stats(group_b1, group_b2, criterion)], ignore_index=True)
    u_stats = pd.concat([u_stats, mann_whitney_u(group_b1, group_b2, criterion)], ignore_index=True)

d_stats.columns = pd.MultiIndex.from_tuples(d_stats.columns)
u_stats.columns = pd.MultiIndex.from_tuples(u_stats.columns)

hyp_B = d_stats.join(u_stats)
hyp_B.index = CRITERIA
display(hyp_B)
print('n = ' + str(len(group_b1)) + ' (Pre-Intervention) & n = ' + str(len(group_b2)) + ' (Post-Intervention)')

Mean       Median      Mode      U-Statistic P-value
          pre  post    pre post  pre post                    
data    0.647 0.750      0    0    0    0     389.000   0.747
methods 1.000 1.500      1    1    1    1     242.000   0.000
results 0.941 1.333      1    1    1    1     288.000   0.002

n = 34 (Pre-Intervention) & n = 24 (Post-Intervention)


For data, there was no statistically significant increase in potential reproducibility at the chosen significance level of 0.01.

For methods and results, there were statistically significant increases in potential reproducibilty at the chosen significance level of 0.01.

We therefore reject the original hypothesis only for the data dimension. 

While the mean ranks have increased for all dimensions, the mode and media stayed the same. 

## Hypothesis C

This step relies on the previous steps to build the groups for the testing. First, the biserial rank scores:  

In [29]:
effect_stats_a = pd.DataFrame()
effect_stats_b = pd.DataFrame()

for criterion in CRITERIA:
    effect_stats_a = pd.concat([effect_stats_a, rank_biserial(group_a1, group_a2, criterion)], ignore_index=True)
    effect_stats_b = pd.concat([effect_stats_b, rank_biserial(group_b1, group_b2, criterion)], ignore_index=True)

effect_stats_a.columns = pd.MultiIndex.from_product([[GROUP_A], effect_stats_a.columns])
effect_stats_b.columns = pd.MultiIndex.from_product([[GROUP_B], effect_stats_b.columns])

hyp_C1 = effect_stats_a.join(effect_stats_b)
hyp_C1.index = CRITERIA

display(hyp_C1)

agile                         giscience                      
        U-Statistic P-value Rank-Biserial U-Statistic P-value Rank-Biserial
data        804.000   0.000         0.456     389.000   0.747         0.047
methods     565.000   0.000         0.618     242.000   0.000         0.407
results     740.000   0.000         0.500     288.000   0.002         0.294

For every tested dimension, the effect size for conference A (AGILE) is larger than for conference B (GIScience). This supports the argument that

- there has been a general increase in potential reproducibility in the research domain over the study period
- the increase is larger in relative terms and absolute ranks for AGILE

Whether this is due to the introduction of the guidelines and the reproducibility reviews is open for interpretation. It should also be noted that there is an overlap in the community (in terms of authors, reviewers, and scientific program committee), so one could expect a certain type of informal spill-over effect. 

Finally, we can also compare the odds ratios for a paper scoring in the category of Available or Open post-intervention:

In [32]:
odds_stats_a = pd.DataFrame()
odds_stats_b = pd.DataFrame()

for criterion in CRITERIA:
    print('\n' + GROUP_A, criterion)
    odds_stats_a = pd.concat([odds_stats_a, odds_ratios(group_a1, group_a2, criterion)], ignore_index=True)
    print('\n' + GROUP_B, criterion)
    odds_stats_b = pd.concat([odds_stats_b, odds_ratios(group_b1, group_b2, criterion)], ignore_index=True)

odds_stats_a.columns = pd.MultiIndex.from_product([[GROUP_A], odds_stats_a.columns])
odds_stats_b.columns = pd.MultiIndex.from_product([[GROUP_B], odds_stats_b.columns])

hyp_C2 = odds_stats_a.join(odds_stats_b)
hyp_C2.index = CRITERIA

display(hyp_C2)


agile data
        high  low
after     32   26
before    11   40

giscience data
        high  low
after      4   20
before     7   27

agile methods
        high  low
after     41   17
before     6   45

giscience methods
        high  low
after     11   13
before     1   33

agile results
        high  low
after     31   27
before     4   47

giscience results
        high  low
after      6   18
before     0   34


agile                           giscience                          
           OR CI lower CI upper P-value        OR CI lower CI upper P-value
data    1.499    0.389    2.608   0.000    -0.260   -2.044    1.525   0.710
methods 2.895    1.552    4.239   0.000     3.329    0.510    6.149   0.000
results 2.602    1.099    4.105   0.000     3.121   -0.744    6.986   0.005

For the data reproducibility dimension, the log odds ratio for an improvement is higher for AGILE than for GScience, which even has a negative log odds ratio. However, the latter is not statistically significant at the chosen level of 0.01. For methods and results, all the log odds ratios for GIScience are slightly higher than for AGILE. This is also an effect of the very low frequency of highly reproducible GIScience papers pre-intervention.  

# Latex table output

This last part only takes the dataframes created above and generates a Latex table from it. 

In [42]:
print(hyp_A.to_latex(float_format="%.3f".__mod__, 
                     escape = True,
                     multicolumn_format = 'c',
                     caption = 'Descriptive statistics and Mann-Whitney-U scores \
values for AGILE group before and after intervention, per criterion'))

\begin{table}
\caption{Descriptive statistics and Mann-Whitney-U scores values for AGILE group before and after intervention, per criterion}
\begin{tabular}{lrrrrrrrr}
\toprule
 & \multicolumn{2}{c}{Mean} & \multicolumn{2}{c}{Median} & \multicolumn{2}{c}{Mode} & U-Statistic & P-value \\
 & pre & post & pre & post & pre & post &  &  \\
\midrule
data & 0.745 & 1.586 & 1 & 2 & 0 & 2 & 804.000 & 0.000 \\
methods & 1.020 & 1.931 & 1 & 2 & 1 & 2 & 565.000 & 0.000 \\
results & 1.020 & 1.741 & 1 & 2 & 1 & 1 & 740.000 & 0.000 \\
\bottomrule
\end{tabular}
\end{table}



In [41]:
print(hyp_B.to_latex(float_format="%.3f".__mod__, 
                     escape = True,
                     multicolumn_format = 'c',
                     caption = 'Descriptive statistics and Mann-Whitney-U scores \
values for GIScience group before and after intervention, per criterion'))

\begin{table}
\caption{Descriptive statistics and Mann-Whitney-U scores values for GIScience group before and after intervention, per criterion}
\begin{tabular}{lrrrrrrrr}
\toprule
 & \multicolumn{2}{c}{Mean} & \multicolumn{2}{c}{Median} & \multicolumn{2}{c}{Mode} & U-Statistic & P-value \\
 & pre & post & pre & post & pre & post &  &  \\
\midrule
data & 0.647 & 0.750 & 0 & 0 & 0 & 0 & 389.000 & 0.747 \\
methods & 1.000 & 1.500 & 1 & 1 & 1 & 1 & 242.000 & 0.000 \\
results & 0.941 & 1.333 & 1 & 1 & 1 & 1 & 288.000 & 0.002 \\
\bottomrule
\end{tabular}
\end{table}



In [37]:
print(hyp_C1.to_latex(float_format="%.3f".__mod__, 
                      escape = True, 
                      multicolumn_format = 'c',
                      caption = 'Rank-Biserial values for AGILE and GIScience groups before and after intervention, per criterion'))

\begin{table}
\caption{Rank-Biserial values for AGILE and GIScience groups before and after intervention, per criterion}
\begin{tabular}{lrrrrrr}
\toprule
 & \multicolumn{3}{c}{agile} & \multicolumn{3}{c}{giscience} \\
 & U-Statistic & P-value & Rank-Biserial & U-Statistic & P-value & Rank-Biserial \\
\midrule
data & 804.000 & 0.000 & 0.456 & 389.000 & 0.747 & 0.047 \\
methods & 565.000 & 0.000 & 0.618 & 242.000 & 0.000 & 0.407 \\
results & 740.000 & 0.000 & 0.500 & 288.000 & 0.002 & 0.294 \\
\bottomrule
\end{tabular}
\end{table}



In [39]:
print(hyp_C2.to_latex(float_format="%.3f".__mod__, 
                      escape = True,
                      multicolumn_format = 'c',
                      caption = 'Odds ratios (OR), upper and lower 95% confidence intervals (CI upper/lower) and Fischer exact P-Values \
for AGILE and GIScience groups before and after intervention, per criterion'))

\begin{table}
\caption{Odds ratios (OR), upper and lower 95% confidence intervals (CI upper/lower) and Fischer exact P-Values for AGILE and GIScience groups before and after intervention, per criterion}
\begin{tabular}{lrrrrrrrr}
\toprule
 & \multicolumn{4}{c}{agile} & \multicolumn{4}{c}{giscience} \\
 & OR & CI lower & CI upper & P-value & OR & CI lower & CI upper & P-value \\
\midrule
data & 1.499 & 0.389 & 2.608 & 0.000 & -0.260 & -2.044 & 1.525 & 0.710 \\
methods & 2.895 & 1.552 & 4.239 & 0.000 & 3.329 & 0.510 & 6.149 & 0.000 \\
results & 2.602 & 1.099 & 4.105 & 0.000 & 3.121 & -0.744 & 6.986 & 0.005 \\
\bottomrule
\end{tabular}
\end{table}

